In [201]:
import numpy as np
import pandas as pd

In [203]:
df = pd.read_csv("/Users/dev/Personal/DS & AI Class Notes/Data Sets/Logistic Regression/diabetes.csv")

In [204]:
df.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

### Func for Checking Zeros

In [205]:
def checkz(df):
    return df[df == 0].value_counts()

In [206]:
for i in df.columns:
    print(checkz(df[i]))

0    111
Name: Pregnancies, dtype: int64
0    5
Name: Glucose, dtype: int64
0    35
Name: BloodPressure, dtype: int64
0    227
Name: SkinThickness, dtype: int64
0    374
Name: Insulin, dtype: int64
0.0    11
Name: BMI, dtype: int64
Series([], Name: DiabetesPedigreeFunction, dtype: int64)
Series([], Name: Age, dtype: int64)
0    500
Name: Outcome, dtype: int64


In [207]:
clist = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']

### Func for Removing Zeros

In [208]:
def zeroremove(df):
    m = round(df.mean(),2)
    df.replace(0,m,inplace = True) 

In [209]:
for i in clist:
    zeroremove(df[i])

### Outcome is already having the  2 output (i.e 0 & 1 ) that's why...i didn't apply the func on Outcome Col.

In [210]:
for i in df.columns:
    print(checkz(df[i]))

Series([], Name: Pregnancies, dtype: int64)
Series([], Name: Glucose, dtype: int64)
Series([], Name: BloodPressure, dtype: int64)
Series([], Name: SkinThickness, dtype: int64)
Series([], Name: Insulin, dtype: int64)
Series([], Name: BMI, dtype: int64)
Series([], Name: DiabetesPedigreeFunction, dtype: int64)
Series([], Name: Age, dtype: int64)
0    500
Name: Outcome, dtype: int64


## OD Tech

In [211]:
def odiqr(df):
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    low = q1 - (1.5 * iqr)
    high = q3 + (1.5 * iqr)
    m = df.mean()
    df = df.apply(lambda x : m if x < low else (m if x > high else x ) )
    return df

In [212]:
def odmsd(df):
    m = round(df.mean(),2)
    s = round(df.std(),2)
    low = round(m-(3*s),2)
    high = round(m+(3*s),2)
    ft1 = df[df<low]
    ft2 = df[df>high]
    df = df.map(lambda x : low if x < low else (high if x > high else x ))

### Checking Skew of Each Columns

In [213]:
for i in df.columns:
    print(f'{i} is {df[i].skew()}')

Pregnancies is 1.0413081819820824
Glucose is 0.5332276258075371
BloodPressure is 0.17300471264838402
SkinThickness is 1.2267253970099732
Insulin is 3.2918295090631706
BMI is 0.6011186113949238
DiabetesPedigreeFunction is 1.919911066307204
Age is 1.1295967011444805
Outcome is 0.635016643444986


### Applying Func for Skew

In [214]:
for i in df.columns:
    if df[i].skew() <= 0.5:
        odmsd(df[i])
    else:
        df[i] = odiqr(df[i])

### After applying IQR on clist Columns the Skew is...


In [215]:
for i in df.columns:
    print(f'{i} is {df[i].skew()}')

Pregnancies is 0.8364127708246099
Glucose is 0.5332276258075371
BloodPressure is 0.17300471264838402
SkinThickness is 0.6153800987519674
Insulin is 1.0069907744811908
BMI is 0.25673836714268067
DiabetesPedigreeFunction is 0.912098278969851
Age is 1.0283798090717455
Outcome is 0.635016643444986


In [216]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.00,148.0,72.0,35.00,79.8,33.6,0.627000,50.0,1
1,1.00,85.0,66.0,29.00,79.8,26.6,0.351000,31.0,0
2,8.00,183.0,64.0,20.54,79.8,23.3,0.672000,32.0,1
3,1.00,89.0,66.0,23.00,94.0,28.1,0.167000,21.0,0
4,3.85,137.0,40.0,35.00,168.0,43.1,0.471876,33.0,1
...,...,...,...,...,...,...,...,...,...
763,10.00,101.0,76.0,48.00,180.0,32.9,0.171000,63.0,0
764,2.00,122.0,70.0,27.00,79.8,36.8,0.340000,27.0,0
765,5.00,121.0,72.0,23.00,112.0,26.2,0.245000,30.0,0
766,1.00,126.0,60.0,20.54,79.8,30.1,0.349000,47.0,1


In [217]:
X = df.drop("Outcome",axis=1)

In [218]:
X.sample()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
463,5.0,88.0,78.0,30.0,79.8,27.6,0.258,37.0


In [219]:
y = df["Outcome"]

In [220]:
y.sample()

399    1
Name: Outcome, dtype: int64

In [325]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold , cross_val_score
from sklearn.linear_model import LogisticRegression , LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier

In [326]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=.20)

In [327]:
X.shape , Xtrain.shape , Xtest.shape

((768, 8), (614, 8), (154, 8))

In [328]:
y.shape , ytrain.shape , ytest.shape

((768,), (614,), (154,))

In [329]:
kf = KFold(n_splits=11)

In [330]:
logreg = LogisticRegression(max_iter=1000)

In [331]:
linreg = LinearRegression()

In [332]:
sv = SVC()

In [333]:
dct = DecisionTreeClassifier()

In [334]:
knn = KNeighborsClassifier()

In [335]:
mul = MultinomialNB()

In [336]:
algo = [logreg , linreg , sv , dct , knn ]

### Without Feature Scaling

In [337]:
for i in algo:
    i.fit(Xtrain,ytrain)
    s = i.score(Xtest,ytest)
    print(f'{i} = {s}')

LogisticRegression(max_iter=1000) = 0.7402597402597403
LinearRegression() = 0.29610955674019845
SVC() = 0.7272727272727273
DecisionTreeClassifier() = 0.6818181818181818
KNeighborsClassifier() = 0.7337662337662337


### With Feature Scaling

In [338]:
ss = StandardScaler()

In [339]:
ss.fit(Xtrain)

StandardScaler()

In [340]:
Xtrain_ss = ss.transform(Xtrain)

In [341]:
Xtest_ss = ss.transform(Xtest)

In [342]:
for i in algo:
    i.fit(Xtrain_ss,ytrain)
    s = i.score(Xtest_ss,ytest)
    print(f'{i} = {s}')

LogisticRegression(max_iter=1000) = 0.7467532467532467
LinearRegression() = 0.29610955674019845
SVC() = 0.7272727272727273
DecisionTreeClassifier() = 0.7077922077922078
KNeighborsClassifier() = 0.7532467532467533


### With Cross Validation

In [343]:
for i in algo:
    s = cross_val_score(i,X,y,cv = kf)
    print(f'{i} = {s.mean()}')

LogisticRegression(max_iter=1000) = 0.7669301712779973
LinearRegression() = 0.28399915973128154
SVC() = 0.7474684735554301
DecisionTreeClassifier() = 0.6836250705815924
KNeighborsClassifier() = 0.7187088274044796


## With Boosting 

In [344]:
from sklearn.ensemble import RandomForestClassifier , BaggingClassifier , AdaBoostClassifier , GradientBoostingClassifier 

In [345]:
rfc = RandomForestClassifier(n_estimators=150,max_depth=3,max_leaf_nodes=4)

In [346]:
baggc = BaggingClassifier(base_estimator=DecisionTreeClassifier())

In [347]:
adbst = AdaBoostClassifier()

In [348]:
gdbst = GradientBoostingClassifier()

In [349]:
algo1 = [rfc , baggc , adbst , gdbst]

In [350]:
for i in algo1:
    print(i)

RandomForestClassifier(max_depth=3, max_leaf_nodes=4, n_estimators=150)
BaggingClassifier(base_estimator=DecisionTreeClassifier())
AdaBoostClassifier()
GradientBoostingClassifier()


### Without Feature Scaling

In [351]:
for i in algo1:
    i.fit(Xtrain,ytrain)
    s = i.score(Xtest,ytest)
    print(f'{i} = {s}')

RandomForestClassifier(max_depth=3, max_leaf_nodes=4, n_estimators=150) = 0.7467532467532467
BaggingClassifier(base_estimator=DecisionTreeClassifier()) = 0.7142857142857143
AdaBoostClassifier() = 0.7662337662337663
GradientBoostingClassifier() = 0.7467532467532467


### With Feature Scaling

In [352]:
for i in algo1:
    i.fit(Xtrain_ss,ytrain)
    s = i.score(Xtest_ss,ytest)
    print(f'{i} = {s}')

RandomForestClassifier(max_depth=3, max_leaf_nodes=4, n_estimators=150) = 0.7077922077922078
BaggingClassifier(base_estimator=DecisionTreeClassifier()) = 0.7467532467532467
AdaBoostClassifier() = 0.7662337662337663
GradientBoostingClassifier() = 0.7532467532467533


###  With Cross Validation (Boosting)

In [353]:
for i in algo1:
    s = cross_val_score(i,X,y,cv = kf)
    print(f'{i} = {s.mean()}')

RandomForestClassifier(max_depth=3, max_leaf_nodes=4, n_estimators=150) = 0.7408808582721627
BaggingClassifier(base_estimator=DecisionTreeClassifier()) = 0.7368530020703933
AdaBoostClassifier() = 0.7344814605684171
GradientBoostingClassifier() = 0.7460756634669679


### With GridSearch CV

In [354]:
from sklearn.model_selection import GridSearchCV ,RandomizedSearchCV 

In [355]:
dic = { 'criterion' : ['gini','entropy'] , 'splitter' : ["best", "random"] , 'max_depth' : [5,9,15,20,25] }

In [356]:
kf1 = KFold(n_splits=20)

### gvc Using without Feature Scaling

In [357]:
gvc = GridSearchCV(DecisionTreeClassifier(),param_grid=dic,cv = kf1)

In [358]:
gvc.fit(X,y)

GridSearchCV(cv=KFold(n_splits=20, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 9, 15, 20, 25],
                         'splitter': ['best', 'random']})

In [359]:
gvc.best_params_

{'criterion': 'gini', 'max_depth': 5, 'splitter': 'random'}

In [360]:
gvc.best_estimator_

DecisionTreeClassifier(max_depth=5, splitter='random')

In [361]:
gvc.best_score_

0.7489541160593791

### gvc Using with Feature Scaling


In [362]:
ss1 = StandardScaler()
X_ss1 = ss1.fit_transform(X)

In [363]:
gvc1 = GridSearchCV(DecisionTreeClassifier(),param_grid=dic,cv = kf1)

In [364]:
gvc1.fit(X_ss1,y)

GridSearchCV(cv=KFold(n_splits=20, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 9, 15, 20, 25],
                         'splitter': ['best', 'random']})

In [365]:
gvc1.best_params_

{'criterion': 'entropy', 'max_depth': 5, 'splitter': 'best'}

In [366]:
gvc1.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [367]:
gvc1.best_score_

0.7323211875843454

### With Random Search CV

In [368]:
rsc =  RandomizedSearchCV(DecisionTreeClassifier(),param_distributions=dic,cv = kf1)

### rsc Using without Feature Scaling

In [369]:
rsc.fit(X,y)

RandomizedSearchCV(cv=KFold(n_splits=20, random_state=None, shuffle=False),
                   estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 9, 15, 20, 25],
                                        'splitter': ['best', 'random']})

In [370]:
rsc.best_params_

{'splitter': 'random', 'max_depth': 5, 'criterion': 'entropy'}

In [371]:
rsc.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=5, splitter='random')

In [372]:
rsc.best_score_

0.7464237516869096

### rsc Using with Feature Scaling



In [373]:
rsc1 =  RandomizedSearchCV(DecisionTreeClassifier(),param_distributions=dic,cv = kf1)

In [374]:
rsc1.fit(X_ss1,y)

RandomizedSearchCV(cv=KFold(n_splits=20, random_state=None, shuffle=False),
                   estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 9, 15, 20, 25],
                                        'splitter': ['best', 'random']})

In [375]:
rsc.best_params_

{'splitter': 'random', 'max_depth': 5, 'criterion': 'entropy'}

In [376]:
rsc.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=5, splitter='random')

In [377]:
rsc.best_score_

0.7464237516869096